<a href="https://colab.research.google.com/github/shivang-niranjan/Hyperspectral-imaging/blob/main/Shivang_Project_2_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!kaggle datasets download -d ssismasterchief/machine-hack-fake-news-content-detection
!unzip machine-hack-fake-news-content-detection.zip -d data

Dataset URL: https://www.kaggle.com/datasets/ssismasterchief/machine-hack-fake-news-content-detection
License(s): unknown
  0% 0.00/560k [00:00<?, ?B/s]
100% 560k/560k [00:00<00:00, 51.4MB/s]
Archive:  machine-hack-fake-news-content-detection.zip
  inflating: data/Test.csv           
  inflating: data/Train.csv          
  inflating: data/sample submission.csv  


In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from tqdm import tqdm
tqdm.pandas()

In [ ]:
data = pd.read_csv("/content/data/Train.csv")

In [ ]:
print(data.shape)
data.head()

(10240, 3)


,Labels,Text,Text_Tag
0,1,Says the Annies List political group supports ...,abortion
1,2,When did the decline of coal start? It started...,"energy,history,job-accomplishments"
2,3,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy
3,1,Health care reform legislation is likely to ma...,health-care
4,2,The economic turnaround started at the end of ...,"economy,jobs"


In [ ]:
data['Original Text'] = data['Text']
print("Original text: ")

print(data['Text'][0])

Original text: 
Says the Annies List political group supports third-trimester abortions on demand.


In [ ]:
import nltk
nltk.download('stopwords')

data["Text"] = data["Text"].str.lower()
stopwords_ = set(stopwords.words('english'))

def stopwords_removal(text):
    return " ".join([word for word in str(text).split() if word not in stopwords_])

tqdm.pandas()
data['Text'] = data['Text'].progress_apply(lambda text: stopwords_removal(text))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
100%|██████████| 10240/10240 [00:00<00:00, 112338.12it/s]


In [ ]:
print("After removing stopwords: ")

print(data['Text'][0])

After removing stopwords: 
says annies list political group supports third-trimester abortions demand.


In [ ]:
punctuations= list(string.punctuation)

def cleaning_punctuations(text):
    return text.translate(str.maketrans('', '', string.punctuation))

data['Text'] = data['Text'].progress_apply(lambda text: cleaning_punctuations(text))

100%|██████████| 10240/10240 [00:00<00:00, 139904.40it/s]


In [ ]:
print("After removing punctuations: ")

print(data['Text'][0])

After removing punctuations: 
says annies list political group supports thirdtrimester abortions demand


In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
num_classes = (data['Labels'].nunique())
X = data['Text']
y = tf.keras.utils.to_categorical(data['Labels'], num_classes)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
train_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
train_tokenizer.fit_on_texts(X_train.values)
train_word_index = train_tokenizer.word_index

In [ ]:
train_tokenizer

In [ ]:
vocab_length = len(train_word_index) + 1
vocab_length

12058

In [ ]:
train_sequences = train_tokenizer.texts_to_sequences(X_train)
len(train_sequences)

8192

In [ ]:
test_sequences = train_tokenizer.texts_to_sequences(X_test)
len(test_sequences)

2048

In [ ]:
maxlen=20
train_padded_seqeunces = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, padding='post', maxlen=maxlen, truncating='post')
test_padded_seqeunces = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, padding='post', maxlen=maxlen, truncating='post')

In [ ]:
embedding_dim = 6
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_length, embedding_dim, input_length=maxlen),
    tf.keras.layers.SimpleRNN(64, activation='tanh', return_sequences=True),
    tf.keras.layers.SimpleRNN(32, activation='tanh', return_sequences=True),
    tf.keras.layers.SimpleRNN(16, activation='tanh'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 6)             72348     
                                                                 
 simple_rnn (SimpleRNN)      (None, 20, 64)            4544      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 20, 32)            3104      
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 16)                784       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense (Dense)               (None, 24)                408       
                                                                 
 dense_1 (Dense)             (None, 6)                 1

In [ ]:
his = model.fit(train_padded_seqeunces, y_train, epochs=5, validation_data=(test_padded_seqeunces, y_test))

Epoch 1/5
256/256 [==============================] - 9s 19ms/step - loss: 1.7730 - accuracy: 0.1904 - val_loss: 1.7549 - val_accuracy: 0.2217
Epoch 2/5
256/256 [==============================] - 4s 17ms/step - loss: 1.7563 - accuracy: 0.2162 - val_loss: 1.7645 - val_accuracy: 0.1963
Epoch 3/5
256/256 [==============================] - 6s 24ms/step - loss: 1.7099 - accuracy: 0.2616 - val_loss: 1.7963 - val_accuracy: 0.2007
Epoch 4/5
256/256 [==============================] - 4s 18ms/step - loss: 1.5490 - accuracy: 0.3583 - val_loss: 1.9410 - val_accuracy: 0.1797
Epoch 5/5
256/256 [==============================] - 4s 17ms/step - loss: 1.2943 - accuracy: 0.4930 - val_loss: 2.1340 - val_accuracy: 0.1816
